# Drive and GPU Connection

Run to connect with google drive to access data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Verify GPU

In [0]:
import tensorflow as tf 
tf.test.gpu_device_name()

'/device:GPU:0'

# Data preprocess and visualization

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Reshape, Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D
from tensorflow.keras.layers import LeakyReLU, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras import backend as K

def load_data(filename):
    with np.load(filename) as f:
        x_train, x_test = f['arr_0'], f['arr_1']
        y_train, y_test = f['arr_2'], f['arr_3']
        return (x_train, y_train), (x_test, y_test)

def preprocess_data(data, feature_range=(-1.0, 1.0)):
    (x_train, y_train), (x_test, y_test) = data
    min, max = feature_range

    x_train = x_train.reshape(-1, 160, 160, 3).astype(np.float32)
    x_train = x_train/255.0
    x_train = x_train * (max - min) + min

    x_test = x_test.reshape(-1, 160, 160, 3).astype(np.float32)
    x_test = x_test/255.0
    x_test = x_test * (max - min) + min

    y_train = y_train.reshape(-1, 160, 160, 3).astype(np.float32)
    y_train = y_train/255.0
    y_train = y_train * (max - min) + min

    y_test = y_test.reshape(-1, 160, 160, 3).astype(np.float32)
    y_test = y_test/255.0
    y_test = y_test * (max - min) + min

    print("Values after preprocessing, min: " + str(x_train.min()) + ", max: " + str(x_train.max()))
    return (x_train, y_train), (x_test, y_test)

def add_mirrored_data(data):
    (x_train, y_train), (x_test, y_test) = data

    x_train_mirrored = np.flip(x_train, 2)
    y_train_mirrored = np.flip(y_train, 2)
    x_test_mirrored = np.flip(x_test, 2)
    y_test_mirrored = np.flip(y_test, 2)

    x_train = np.concatenate((x_train, x_train_mirrored))
    y_train = np.concatenate((y_train, y_train_mirrored))
    x_test = np.concatenate((x_test, x_test_mirrored))
    y_test = np.concatenate((y_test, y_test_mirrored))

    return (x_train, y_train), (x_test, y_test)

def get_random_batch(data, batch_size):
    idx = np.random.choice(data.shape[0], batch_size)
    batch = data[idx].reshape((batch_size, 160, 160, 3))
    return batch

def show_images(images, feature_range=(-1.0, 1.0)):
    min, max = feature_range
    plt.figure(figsize=(10,10))
    
    for im in range(images.shape[0]):
      plt.subplot(4, 4, im+1)
      image = images[im, :, :, :]
      image = np.reshape(image, [160,160,3])
      image = (image - min) / (max - min)
      plt.imshow(image)
      plt.axis('off')
    plt.tight_layout()
    plt.show()

def visualize(sample_X, sample_Y, encoder, decoder, feature_range=(-1.0, 1.0)):
    min, max = feature_range
    count = sample_X.shape[0]

    reconstructed_X = []
    reconstructed_Y = []
    accumulated = []

    for n in range(count):
        code = encoder.predict(sample_X[n][None])[0]
        reco = decoder.predict(code[None])[0]

        reconstructed_X.append(reco)

        code = encoder.predict(sample_Y[n][None])[0]
        reco = decoder.predict(code[None])[0]

        reconstructed_Y.append(reco)

    for i in range(4):
        accumulated.append(sample_X[i])
        accumulated.append(reconstructed_X[i])
        accumulated.append(sample_Y[i])
        accumulated.append(reconstructed_Y[i])

    count = len(accumulated)

    plt.figure(figsize=(8,8))

    for i in range(count):
        plt.subplot(4, 4, i + 1)
        image = (accumulated[i] - min) / (max - min)
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

Using TensorFlow backend.


# VAE networks

In [0]:
def sampling(args):
    mean_mu, log_var = args
    epsilon = tf.random.normal(shape=K.shape(mean_mu), mean=0., stddev=1.) 
    return mean_mu + K.exp(log_var/2)*epsilon

def build_encoder(img_shape, code_size):
    encoder_input = Input(shape=img_shape, name='encoder_input')

    x = encoder_input

    x = Conv2D(filters=32, kernel_size=(4, 4), strides=(2, 2), padding='same', name='encoder_conv_1')(x)
    x = LeakyReLU()(x)

    x = Conv2D(filters=64, kernel_size=(4, 4), strides=(2, 2), padding='same', name='encoder_conv_2')(x)
    x = LeakyReLU()(x)

    x = Conv2D(filters=128, kernel_size=(4, 4), strides=(2, 2), padding='same', name='encoder_conv_3')(x)
    x = LeakyReLU()(x)

    x = Conv2D(filters=256, kernel_size=(4, 4), strides=(2, 2), padding='same', name='encoder_conv_4')(x)
    x = LeakyReLU()(x)

    x = Flatten()(x)

    mean_mu = Dense(code_size, name='mean_mu')(x)
    log_var = Dense(code_size, name='log_var')(x)
    encoder_output = Lambda(sampling, name='encoder_output')([mean_mu, log_var])

    return encoder_input, encoder_output, mean_mu, log_var, Model(encoder_input, encoder_output)

def build_decoder(img_shape, code_size):
    size1 = int(img_shape[0]/16) # 8 is 2^Conv2D in encoder
    size2 = int(img_shape[1]/16)

    decoder_input = Input(shape=(int(code_size,)), name='decoder_input')

    x = decoder_input

    x = Dense(units=(size1*size2*256))(x)
    x = Reshape(target_shape=(size1, size2, 256))(x)
    
    x = Conv2DTranspose(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='decoder_conv_1')(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same', name='decoder_conv_2')(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same', name='decoder_conv_3')(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(filters=3, kernel_size=(3, 3), strides=(2, 2), padding='same', name='decoder_conv_4')(x)
    x = Activation('tanh')(x)

    decoder_output = x

    return decoder_input, decoder_output, Model(decoder_input, decoder_output)

# Training

In [0]:
tf.compat.v1.disable_eager_execution()
# Load and prepare data
npz_path = './drive/My Drive/ColabStuff/DiCaprioToDowneyJr_Big_VAE.npz'
(x_train, y_train), (x_test, y_test) = load_data(npz_path)

value_range = (-1.0, 1.0)
print("Imported data shape: " + str(x_train.shape))

(x_train, y_train), (x_test, y_test) = preprocess_data(((x_train, y_train), (x_test, y_test)), value_range)

# imported_data_sample = get_random_batch(x_train, 16)
# show_images(imported_data_sample, value_range)

(x_train, y_train), (x_test, y_test) = add_mirrored_data(((x_train, y_train), (x_test, y_test)))

# imported_data_sample = get_random_batch(x_train, 16)
# show_images(imported_data_sample, value_range)

save_path = './drive/My Drive/ColabStuff/VAE/'

train_x = np.concatenate((x_train, x_test))
train_y = np.concatenate((y_train, y_test))

# # # # # # VAE training 1 # # # # # #

IMG_SHAPE = x_train.shape[1:]
code_size = 200
epochs_number_1 =150
epochs_number_2 =800
LEARNING_RATE = 0.0005
LOSS_FACTOR = 1000

vae_encoder_input, vae_encoder_output, mean_mu, log_var, vae_encoder = build_encoder(IMG_SHAPE, code_size)
# print(vae_encoder.summary())

vae_decoder_input, vae_decoder_output, vae_decoder = build_decoder(IMG_SHAPE, code_size)
# print(vae_decoder.summary())

vae_input = vae_encoder_input
vae_output = vae_decoder(vae_encoder_output)
vae_model = Model(vae_input, vae_output)


def r_loss(y_true, y_pred):
    r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
    return r_loss

def kl_loss(y_true, y_pred):
    kl_loss =  -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean_mu) - tf.exp(log_var), axis = 1)
    return kl_loss

def total_loss(y_true, y_pred):
    return LOSS_FACTOR*r_loss(y_true, y_pred) + kl_loss(y_true, y_pred)
  
adam_optimizer = Adam(lr = LEARNING_RATE)

vae_model.compile(optimizer=adam_optimizer, loss = total_loss, metrics = ['accuracy', r_loss, kl_loss])

batch_size = 128

loss_dis = []
r_loss_plot = []
kl_loss_plot = []
acc_dis = []
plot_iteration = []

for i in range(1, epochs_number_1 + 1):
    images_train_x = get_random_batch(train_x, batch_size)
    images_train_y = get_random_batch(train_y, batch_size)

    train_z = np.concatenate((images_train_x, images_train_y))

    d_stats = vae_model.train_on_batch(x= train_z, y= train_z)
    loss_dis.append(d_stats[0])
    acc_dis.append(d_stats[1])
    r_loss_plot.append(d_stats[2])
    kl_loss_plot.append(d_stats[3])
    plot_iteration.append(i)

    if (i != 1) and (i % 10 == 0):
      print('setp: ' + str(i) + ", r_loss: " + str(r_loss_plot[i-1]) + ", kl_loss: " + str(kl_loss_plot[i-1]) + ", total_loss: " + str(loss_dis[i-1]))

    if (i !=1)and (i % 50 == 0):

      (fig, (ax1)) = plt.subplots(1)
      fig.set_size_inches(8, 8)

      ax1.plot(plot_iteration, loss_dis, label="total loss")
      ax1.legend()

      ax1.plot(plot_iteration, r_loss_plot, "C1", label="r_loss")
      ax1.legend()

      ax1.plot(plot_iteration, kl_loss_plot, "C2", label="kl_loss")
      ax1.legend()

      plt.show()

      (fig, (ax1)) = plt.subplots(1)
      fig.set_size_inches(8, 8)

      ax1.plot(plot_iteration, acc_dis, "C1", label="accuracy")
      ax1.legend()

      plt.show()
          
    if (i !=1) and (i % 50 == 0):

      test_sample_X = get_random_batch(train_x, 4)
      test_sample_Y = get_random_batch(train_y, 4)

      visualize(test_sample_X, test_sample_Y, vae_encoder, vae_decoder, value_range)

vae_encoder.save(save_path + "encoder_" + str(i) + ".h5")
vae_decoder.save(save_path + "decoder_XY_" + str(i) + ".h5")

layer_number = 0
for layer in vae_encoder.layers:
  layer_number += 1
  print("Layer: " + str(layer_number))
  layer.trainable = False

# # # # # # VAE end training 1 # # # # # #

# # # # # # VAE training 2 # # # # # #

vae2_decoder_input, vae2_decoder_output, vae2_decoder = build_decoder(IMG_SHAPE, code_size)
print(vae2_decoder.summary())

vae2_output = vae2_decoder(vae_encoder_output)
vae2_model = Model(vae_input, vae2_output)

vae2_model.compile(optimizer=adam_optimizer, loss = total_loss, metrics = ['accuracy'])

loss_dis = []
acc_dis = []
plot_iteration = []

for i in range(1, epochs_number_2 + 1):
    images_train_y = get_random_batch(train_y, batch_size)

    d_stats = vae2_model.train_on_batch(x= images_train_y, y= images_train_y)
    loss_dis.append(d_stats[0])
    acc_dis.append(d_stats[1])
    plot_iteration.append(i)

    if (i !=1)and (i % 50 == 0):

      (fig, (ax1)) = plt.subplots(1)
      fig.set_size_inches(8, 8)

      ax1.plot(plot_iteration, loss_dis, label="loss discriminator")
      ax1.legend()

      plt.show()

      (fig, (ax1)) = plt.subplots(1)
      fig.set_size_inches(8, 8)

      ax1.plot(plot_iteration, acc_dis, "C1", label="acc discriminator")
      ax1.legend()

      plt.show()
          
    if (i !=1) and (i % 50 == 0):

      test_sample_X = get_random_batch(train_x, 4)
      test_sample_Y = get_random_batch(train_y, 4)

      visualize(test_sample_X, test_sample_Y, vae_encoder, vae2_decoder, value_range)

    if (i > 99) and (i % 50 == 0):
      vae2_decoder.save(save_path + "decoder_Y_" + str(i) + ".h5")

# # # # # # # VAE end training 2 # # # # # #

# # # # # # VAE training 3 # # # # # #

vae3_decoder_input, vae3_decoder_output, vae3_decoder = build_decoder(IMG_SHAPE, code_size)
print(vae3_decoder.summary())

vae3_output = vae3_decoder(vae_encoder_output)
vae3_model = Model(vae_input, vae3_output)

vae3_model.compile(optimizer=adam_optimizer, loss = total_loss, metrics = ['accuracy'])

loss_dis = []
acc_dis = []
plot_iteration = []

for i in range(1, epochs_number_2 + 1):
    images_train_x = get_random_batch(train_x, batch_size)

    d_stats = vae3_model.train_on_batch(x= images_train_x, y= images_train_x)
    loss_dis.append(d_stats[0])
    acc_dis.append(d_stats[1])
    plot_iteration.append(i)


    if (i !=1)and (i % 50 == 0):

      (fig, (ax1)) = plt.subplots(1)
      fig.set_size_inches(8, 8)

      ax1.plot(plot_iteration, loss_dis, label="loss discriminator")
      ax1.legend()

      plt.show()

      (fig, (ax1)) = plt.subplots(1)
      fig.set_size_inches(8, 8)

      ax1.plot(plot_iteration, acc_dis, "C1", label="acc discriminator")
      ax1.legend()

      plt.show()
          
    if (i !=1) and (i % 50 == 0):

      test_sample_X = get_random_batch(train_x, 4)
      test_sample_Y = get_random_batch(train_y, 4)

      visualize(test_sample_X, test_sample_Y, vae_encoder, vae3_decoder, value_range)

    if (i > 99) and (i % 50 == 0):
      vae3_decoder.save(save_path + "decoder_X_" + str(i) + ".h5")

# # # # # # # VAE end training 3 # # # # # #

print("Program finished successfully!")

Output hidden; open in https://colab.research.google.com to view.